# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Model
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

experiment_name = 'hyperdrive_heart_failure_prediction'
# project_folder = './capstone-project/hyperdrive'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
hyperdrive_heart_failure_prediction,quick-starts-ws-137804,Link to Azure Machine Learning studio,Link to Documentation


In [5]:


compute_cluster_name = "capstone-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace = ws, name = compute_cluster_name)
    print('Found existing cluster, use it.')
    
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded..............
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [8]:
# Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 1, delay_evaluation = 5)

# Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "--C" : choice(0.1, 1),
    "--max_iter" : choice(50, 100, 150, 200)
    })


# Create your estimator and hyperdrive config

# if "training" not in os.listdir():
#     os.mkdir("./training")
    
estimator = SKLearn(source_directory = '.', entry_script = 'train.py', compute_target = compute_cluster_name)

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling = param_sampling,
                                     primary_metric_name = "Accuracy",
                                     primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs = 10,
                                     max_concurrent_runs = 4,
                                     policy = early_termination_policy,
                                     estimator = estimator)

In [11]:
from azureml.core import Dataset
from azureml.core import Workspace

subscription_id = 'f9d5a085-54dc-4215-9ba6-dad5d86e60a0'
resource_group = 'aml-quickstarts-137804'
workspace_name = 'quick-starts-ws-137804'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='Heart_Failure_Clinical_Records_Dataset')

df = dataset.to_pandas_dataframe()

In [12]:
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [9]:
# Submit your experiment

hyperdrive_run = experiment.submit(hyperdrive_run_config)

RunDetails(hyperdrive_run).show()

hyperdrive_run.get_status()

hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_5d8df2da-e2e2-4ef8-8b85-9d4bf91748f8
Web View: https://ml.azure.com/experiments/hyperdrive_heart_failure_prediction/runs/HD_5d8df2da-e2e2-4ef8-8b85-9d4bf91748f8?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-137804/workspaces/quick-starts-ws-137804

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-02-07T10:59:54.961588][API][INFO]Experiment created<END>\n""<START>[2021-02-07T10:59:55.412337][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-02-07T10:59:55.572045][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-02-07T10:59:56.0608207Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_5d8df2da-e2e2-4ef8-8b85-9d4bf91748f8
Web View: https://ml.azure.com/experiments/hyperdrive_heart_failure_prediction/runs/HD_5d8df2da-e2e2-4ef8-8b8

{'runId': 'HD_5d8df2da-e2e2-4ef8-8b85-9d4bf91748f8',
 'target': 'capstone-cluster',
 'status': 'Canceled',
 'startTimeUtc': '2021-02-07T10:59:54.744793Z',
 'endTimeUtc': '2021-02-07T11:23:21.398607Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_metric_config': '{"name"

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:

# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print('best run',best_run)

best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n learning rate:', parameter_values)

In [ ]:
#TODO: Save the best model
joblib.dump(parameter_values, filename='output/best-hyperdrive.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model = Model.register(workspace = ws, model_name ='best-hyperdrive',
                       model_path = './output/best-hyperdrive.pkl',
                       tags = {'Method' : 'HyperDrive'},
                       properties={'Accuracy': best_run_metrics['Accuracy']})

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service